In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)


tensor([[ 0.0833, -0.1919,  0.0514, -0.1303,  0.0376, -0.1334, -0.3882, -0.0726,
         -0.1599,  0.0107],
        [ 0.0536, -0.2340,  0.1119, -0.0059,  0.0550, -0.0409, -0.2969, -0.0581,
         -0.0532,  0.0726]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)


tensor([[-0.0297,  0.0817,  0.0356,  0.0642, -0.0052,  0.3338, -0.1863,  0.2744,
         -0.0657, -0.1331],
        [ 0.0164,  0.1578, -0.0769,  0.0565,  0.1303,  0.2584, -0.1031,  0.1231,
         -0.0876, -0.0326]], grad_fn=<AddmmBackward0>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [5]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1761, -0.0776, -0.1563,  0.1269,  0.1169, -0.1040, -0.0133,  0.1557,
          0.2448,  0.0408],
        [-0.1210, -0.0474, -0.1175,  0.0514,  0.0845, -0.0325, -0.1185,  0.1706,
          0.2502,  0.0024]], grad_fn=<AddmmBackward0>)

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [7]:
net = FixedHiddenMLP()
net(X)


tensor(0.0445, grad_fn=<SumBackward0>)

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)


tensor(-0.1273, grad_fn=<SumBackward0>)

In [9]:
!nvidia-smi


'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.
